In [1]:
# Importing libraries
from copy import deepcopy
import random


# Math Libraries
import random
import math
import numpy as np

# Data Processing
import pandas as pd

# Libraries for data visualization
import matplotlib.pyplot as plt  
import seaborn as sns 
from sympy import var, plot_implicit

# ML
from sklearn.linear_model import LogisticRegression # Importing Logistic Model
from sklearn.model_selection import train_test_split # Train Test Split
from sklearn.preprocessing import MinMaxScaler # Data normalizer Min Max Scale
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error # Calculate the accuracy

# Loading Bar
from time import sleep
from tqdm import tqdm

# Convex Hull
from scipy.spatial import ConvexHull, convex_hull_plot_2d, Delaunay
from numpy.linalg import det
from scipy.stats import dirichlet
from scipy.spatial.distance import euclidean
from scipy import stats as st
from scipy.optimize import fmin_tnc

# Supress warnings
import warnings; warnings.simplefilter('ignore')

In [2]:
def Adult_Data_Clean(df: pd.DataFrame) -> pd.DataFrame:
  # Changing collumn names for convenience
  df.rename(columns={'capital-gain': 'gain', 'capital-loss': 'loss', 'native-country': 'country',
                    'hours-per-week': 'hours','marital-status': 'marital'}, inplace=True)
  
  # Finding not known data
  df['country'] = df['country'].replace('?',np.nan)
  df['workclass'] = df['workclass'].replace('?',np.nan)
  df['occupation'] = df['occupation'].replace('?',np.nan)
  
  # Dropping not known data
  df.dropna(how='any',inplace=True)

  # Normalizing numerical features
  numerical = ['age', 'fnlwgt', 'educational-num', 'gain', 'loss', 'hours']
  scaler = MinMaxScaler()
  df[numerical] = scaler.fit_transform(df[numerical])  
  return df

In [3]:
# Loading Dataset
adult_folder = pd.read_csv('adult.csv') # Loading Adult

In [4]:
adult_dataset = Adult_Data_Clean(adult_folder)

# Separating label (income) from the rest of the data and making income binary
income_raw = adult_dataset['income'].tolist()
adult_dataset = adult_dataset.drop(['income'], axis=1)
income = pd.Series(income_raw).astype('category').cat.codes.tolist()
adult_dataset.drop(adult_dataset.columns.difference(['age', 'educational-num', 'fnlwgt', 'gender', 'loss', 'hours']), 1, inplace=True)
#adult_dataset.drop(adult_dataset.columns.difference(['educational-num']), 1, inplace=True)
income = np.array(income)

In [5]:
# One-Hot encoding
per_adult_encoded = pd.get_dummies(adult_dataset)

In [6]:
# Spliting dataset
X_train, X_test, Y_train, Y_test = train_test_split(per_adult_encoded, income, test_size = 0.5, random_state = 0)

## Implementing logistic RegressioN

In [7]:
def sigmoid(x):
    # Activation function used to map any real value between 0 and 1
    return 1 / (1 + np.exp(-x))

def net_input(theta, x):
    # Computes the weighted sum of inputs
    return np.dot(x, theta)

def probability(theta, x):
    # Returns the probability after passing through sigmoid
    return sigmoid(net_input(theta, x))

def get_prediction(theta, x):
    return (probability(theta, x) > 0.5)*1

def get_accuracy(theta, x, y):
    return np.mean(get_prediction(theta, x) == y)

In [8]:
def cost_function(theta, x, y):
    # Computes the cost function for all the training samples
    m = x.shape[0]
    total_cost = -(1 / m) * np.sum(
        y * np.log(probability(theta, x)) + (1 - y) * np.log(
            1 - probability(theta, x)))
    return total_cost

def gradient(theta, x, y):
    # Computes the gradient of the cost function at the point theta
    m = x.shape[0]
    return (1 / m) * np.dot(x.T, sigmoid(net_input(theta,   x)) - y)

In [9]:
def fit(x, y, theta):
    opt_weights = fmin_tnc(func=cost_function, x0=theta,
                  fprime=gradient,args=(x, y.flatten())) ;
    
    return opt_weights[0]


X = np.c_[np.ones((X_train.shape[0], 1)), X_train]
theta = np.zeros((X.shape[1], 1))
parameters = fit(X, Y_train, theta)

  NIT   NF   F                       GTG
    0    1  6.931471805599452E-01   1.27138251E-01
    1    4  5.381529011666284E-01   2.10061024E-03
tnc: fscale = 21.8186
    2    9  4.400317303121448E-01   1.68707536E-03
    3   13  4.303388966236409E-01   5.32167574E-05
    4   16  4.298443850717018E-01   1.34777221E-05
    5   21  4.294295481980749E-01   1.73029612E-06
tnc: fscale = 760.221
    6   26  4.293851413686517E-01   4.94415404E-09
    7   29  4.293846836360435E-01   2.80137491E-08
    8   34  4.293841402051041E-01   9.65589239E-11
tnc: fscale = 101766
tnc: |fn-fn-1] = 2.01802e-09 -> convergence
    9   37  4.293841381870887E-01   2.86248793E-10
tnc: Converged (|f_n-f_(n-1)| ~= 0)


## Calculating the Rashomon set

### Finding points in the border

In [10]:
center = parameters
delta = 0.1 #Size of initial noise
samples = 1000 #number of directions
epsilon = 0.1  #Rashomon set size

extremes_l = np.zeros((samples, center.size))

early_stopping_exploration = 1000
for i in tqdm(range(samples)):
    #Generationg direction
    Z = np.random.normal(loc=0.0, scale=1.0, size=center.size)
    Z = Z/np.linalg.norm(Z)
    direction = center + delta * Z
    #loading model
    ct = 1
    while (cost_function(direction, X, Y_train) - cost_function(center, X, Y_train)) < epsilon:
        extremes_l[i, :] = direction
        direction = center + ct*delta*Z
        ct += 1
        if ct==early_stopping_exploration:
            break

100%|███████████████████████████████████████| 1000/1000 [01:15<00:00, 13.17it/s]


### Getting convex hull and triangulation

In [12]:
# Sampling from Rashomon set after calculating cvx hull and triangulation
def samp_in_hull_after(deln, vols, n):
    sample = np.random.choice(len(vols), size = n, p = vols / vols.sum())
    return np.einsum('ijk, ij -> ik', deln[sample], dirichlet.rvs([1]*(dims + 1), size = n))

In [13]:
dims = extremes_l.shape[-1] #get dim
hull = extremes_l[ConvexHull(extremes_l).vertices] #get hull
deln = hull[Delaunay(hull).simplices] #get Delunay
vols = np.abs(det(deln[:, :dims, :] - deln[:, dims:, :])) / np.math.factorial(dims) #get areas

### Sampling models in Rashomon

In [75]:
n = 10000 #number of predictions
models_in_ensemble = samp_in_hull_after(deln, vols, n)

## Getting all predictions

In [76]:
def get_all_predictions(models_in_ensemble, x):
    n_models = models_in_ensemble.shape[0]
    pred = np.zeros((n_models, x.shape[0]))
    for i in range(n_models):
        pred[i] = get_prediction(models_in_ensemble[i], x)
    return pred

In [77]:
predictions = get_all_predictions(models_in_ensemble, X)

In [78]:
print(str(100*np.mean(np.mean(predictions, axis = 0) != 0)) + '% of all training data had 1 oor more different predictions')


55.24744593339525% of all training data had 1 oor more different predictions


In [79]:
idx = np.abs(np.mean(predictions, axis = 0) - 0.5) < 0.1
np.mean(np.abs(np.mean(predictions, axis = 0) - 0.5) < 0.1)

0.04935650789438769

In [83]:
np.sort(np.abs(np.mean(predictions, axis = 0) - 0.5))[:10]

array([0.0003, 0.0004, 0.0005, 0.0006, 0.0006, 0.0006, 0.0008, 0.001 ,
       0.001 , 0.001 ])